In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
%%capture
!conda install -c conda-forge openmm mdtraj parmed
!pip install py3dmol 

In [ ]:
import openmm as mm
import openmm.app as app
import openmm.unit as unit
import numpy as np
from sys import stdout
from parmed import Structure, Atom, Bond

### Create Structure and Topology objects

In [ ]:
# System parameters
M = 100   # Number of chains
N = 10    # Atoms per chain
mass = 120.0  # Mass in amu

# Create an empty structure
structure = Structure()

# Add atoms and residues
for m in range(M):
    for n in range(N):
        atom = Atom(name='CG', type='CG', atomic_number=0, mass=mass, charge=0.0)
        resname = f"CG{m}"
        resnum = m + 1
        structure.add_atom(atom=atom, resname=resname, resnum=resnum, chain='A')

# Assign simple positions: spaced along z-axis
positions = []
for m in range(M):
    x0 = np.array(((m%10)*1.0, (m//10)*1.0, 0))
    positions.append(x0)
    for i in range(1,N):
        xi = positions[-1] + np.array((0, 0, 0.38))
        positions.append(xi)

# Convert the list into OpenMM Quantity with units of nm
structure.positions = positions * unit.nanometer

# Add bonds between consecutive atoms in each chain
for m in range(M):
    offset = m * N
    for n in range(N - 1):
        i = offset + n
        structure.bonds.append(Bond(structure.atoms[i], structure.atoms[i + 1]))

# Save the files
structure.save('cg_polymer.pdb', overwrite=True)
structure.save('cg_polymer.psf', overwrite=True)

In [34]:
# Check basic info
print(structure.topology)
structure.to_dataframe().head()

<Topology; 1 chains, 100 residues, 1000 atoms, 900 bonds>


,number,name,type,atomic_number,charge,mass,nb_idx,solvent_radius,screen,occupancy,...,rmin_14,epsilon_14,resname,resid,resnum,chain,segid,xx,xy,xz
0,-1,CG,CG,0,0.0,120.0,0,0.0,0.0,0.0,...,0.0,0.0,CG0,0,1,A,,0.0,0.0,0.0
1,-1,CG,CG,0,0.0,120.0,0,0.0,0.0,0.0,...,0.0,0.0,CG0,0,1,A,,0.0,0.0,3.8
2,-1,CG,CG,0,0.0,120.0,0,0.0,0.0,0.0,...,0.0,0.0,CG0,0,1,A,,0.0,0.0,7.6
3,-1,CG,CG,0,0.0,120.0,0,0.0,0.0,0.0,...,0.0,0.0,CG0,0,1,A,,0.0,0.0,11.4
4,-1,CG,CG,0,0.0,120.0,0,0.0,0.0,0.0,...,0.0,0.0,CG0,0,1,A,,0.0,0.0,15.2


## Defining the force field
We will cover two methods for defining the forcefield:

1. Using the Python API.
2. Creating a forcefield xml file.

Both methods achieve the same result of defining the forcefield. There are pros and cons of both ways. The key points are that using the Python API is more flexible, while creating a forcefield xml file makes it easier to share your forcefield and helps reproducibility. This is demonstrated in the other OpenMM tutorials --- the standard forcefields such as Amber are distributed with OpenMM as xml files, while the custom forces used to do free energy calculations are defined using the Python API.

### Option-1: Defining a force-field using the Python API

We can create instances of the OpenMM [Force classes](http://docs.openmm.org/latest/api-python/library.html#forces), assign parameters, and add them to the system.
First we must create a [System](http://docs.openmm.org/latest/api-python/generated/openmm.openmm.System.html#openmm.openmm.System).


In [35]:
# create the system and add the particles to it
system = mm.System()

for i in range(len(structure.atoms)):
  system.addParticle(structure.atoms[i].mass)

box_vecs = 11 * np.eye(3) *unit.nanometer
system.setDefaultPeriodicBoxVectors(*box_vecs )

In [36]:
harmonic_bond_force = mm.HarmonicBondForce()

# Add each bond to the force from the topology
for bond in structure.topology.bonds():
    harmonic_bond_force.addBond(bond.atom1.index, bond.atom2.index, 0.38, 1000)

# Define a Lennard-Jones potential
expression = '4*epsilon*((sigma/r)^12-(sigma/r)^6);'\
            + ' sigma=0.5*(sigma1+sigma2);'\
            + ' epsilon=sqrt(epsilon1*epsilon2)'

custom_nb_force = mm.CustomNonbondedForce(expression)

custom_nb_force.addPerParticleParameter('sigma')
custom_nb_force.addPerParticleParameter('epsilon')

# Add the parameters for each particle
for atom in structure.topology.atoms():
    custom_nb_force.addParticle([0.5, 1.0])

# Create exclusions for directly bonded atoms
custom_nb_force.createExclusionsFromBonds([(bond[0].index, bond[1].index) for bond in structure.topology.bonds()], 1)

# Set a cutoff of 1.5nm
custom_nb_force.setNonbondedMethod(mm.CustomNonbondedForce.CutoffPeriodic)
custom_nb_force.setCutoffDistance(1.5*unit.nanometers)

# Add the forces to the system
system.addForce(harmonic_bond_force)
system.addForce(custom_nb_force)


1


### Option-2" Creating a force-field xml file

Alternatively, we can create a custom force-field xml file for our system. You should look here first for the full information: http://docs.openmm.org/latest/userguide/application/05_creating_ffs.html.

You will need to create a new file called "cg_ff.xml" using a text editor of your choice. Then paste the following lines into it:

```xml
<!-- cg_ff.xml 
Coarse-grained forcefield for a bead-spring polymer. -->
<ForceField>
	
    <AtomTypes>
	    <Type name="CG-bead" class="CG-bead" element="CG" mass="120.0"/>
	</AtomTypes>

	<Residues>
        <!-- Each bead is a single residue.
        Need a template for the different Residue types. 
        First type is the end. This only has one bond. -->
        <Residue name="CG-residue-end">
            <Atom name="CG-bead" type="CG-bead"/>
            <ExternalBond atomName="CG-bead"/>
        </Residue>

        <!-- Second type is in the middle of the chain. This has two bonds. -->
        <Residue name="CG-residue-middle">
            <Atom name="CG-bead" type="CG-bead"/>
            <ExternalBond atomName="CG-bead"/>
            <ExternalBond atomName="CG-bead"/>
        </Residue>
    </Residues>

    <!-- Define a harmonic bond between the CA-beads -->
    <HarmonicBondForce>
        <Bond class1="CG-bead" class2="CG-bead" length="0.38" k="1000.0"/>
    </HarmonicBondForce>

    <!-- Use a custom non-bonded force for maximum flexibility.
    The bondCutoff=1 tells it to only exclude interactions between directly bonded atoms. -->
    <CustomNonbondedForce energy="4*epsilon*((sigma/r)^12-(sigma/r)^6); sigma=0.5*(sigma1+sigma2); epsilon=sqrt(epsilon1*epsilon2)" bondCutoff="1">
        <PerParticleParameter name="sigma"/>
        <PerParticleParameter name="epsilon"/>
		<Atom type="CG-bead" sigma="0.5" epsilon="1.0"/>
	</CustomNonbondedForce>
</ForceField>
```
(If you have cloned the cookbook repo then this file will already exist)
The comments in the above code explain what the difference sections are for.


### Create the system

We can now load in our previously created custom `ForceField` and use the `createSystem` method with the `topology`.

In [ ]:
# load in the ForceField we created
ff = app.ForceField('cg_ff.xml')
system2 = ff.createSystem(topology, nonbondedMethod=app.CutoffPeriodic, nonbondedCutoff=1.5*unit.nanometers)

We can compare the two systems to check if they are the same. A simple way to do this is to serialize the systems as save them as xml files.

In [ ]:
with open('system1.xml', 'w') as output:
    output.write(mm.XmlSerializer.serialize(system))

with open('system2.xml', 'w') as output:
    output.write(mm.XmlSerializer.serialize(system2))

!diff system1.xml system2.xml

We actually find they are slightly different. `system2` created by `ForceField.createSystem` has a `CMMotionRemover` that was added by default when using the `createSystem` method. If we wanted to we could add this to the first system.

## Setup and run the simulation

We will use a `LangevinMiddleIntegrator` with a friction term of 0.01ps^-1 and a timestep of 0.01ps as used in similar coarse-grained polymer models [1]. For your own models these parameters will be important and you will need to choose them carefully!

In [ ]:
integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 0.01/unit.picosecond, 0.010*unit.picoseconds)
simulation = app.Simulation(structure.topology, system, integrator)
simulation.context.setPositions(positions)

# setup simulation reporters
# Write the trajectory to a file called 'traj.dcd'
simulation.reporters.append(app.DCDReporter('traj.dcd', 1000, enforcePeriodicBox=False))

# Report information to the screen as the simulation runs
simulation.reporters.append(app.StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True, volume=True, speed=True))


# NVT equilibration
simulation.step(10000)

# Add a barostat
barostatIndex=system.addForce(mm.MonteCarloBarostat(1.0*unit.bar, 300*unit.kelvin))
simulation.context.reinitialize(preserveState=True)

# Run NPT equilibration
simulation.step(100000)


#"Step","Potential Energy (kJ/mole)","Temperature (K)","Box Volume (nm^3)","Speed (ns/day)"
1000,-2504.836181640625,191.5762888669726,1331.0,0
2000,-2815.38623046875,228.05667202079903,1331.0,1e+03
3000,-2720.246337890625,237.99379315778745,1331.0,968
4000,-2717.5224609375,246.43023451709598,1331.0,942
5000,-2659.786865234375,253.65382630735783,1331.0,900
6000,-2649.3740234375,262.1551793714623,1331.0,877
7000,-2439.745361328125,257.93077347642696,1331.0,879
8000,-2385.37646484375,261.5250756318168,1331.0,894
9000,-2507.65869140625,277.1601449505948,1331.0,894
10000,-2277.37890625,263.72359415211446,1331.0,888
11000,-2352.224853515625,277.5869879541712,1224.1066608916633,881
12000,-2151.421875,272.3363540797704,1217.1759308796754,850
13000,-2169.27490234375,274.05078020934116,1126.4046429984535,849
14000,-2102.018310546875,272.95167374749195,1059.0785394736092,831
15000,-2159.517578125,280.71576208676345,1019.7380219560032,810
16000,-2180.33447265625,283.8484832712969,916.285214431259,

NameError: name 'topology' is not defined

### Visualize Simulation

In [ ]:
# Animation
view = py3Dmol.view(width=800, height=800)

view.addModelsAsFrames(open('ljtraj.pdb', 'r').read(),'pdb')
view.setBackgroundColor('white')
view.setStyle({'sphere': {'color':'green'}}) 

#
view.zoomTo()
view.animate({'loop': "forward"})
view.show()

In [40]:
import mdtraj as md
import nglview as nv
from IPython.display import display

# Load trajectory with PSF as topology
traj = md.load('traj.dcd', top='equilibrated_config.pdb')

# Visualize with connectivity
view = nv.show_mdtraj(traj)
view.add_representation('ball+stick', selection='all')  # or 'licorice'

# Display the viewer
display(view)


NGLWidget(max_frame=109)